# Project: Markov Simulation

## Business goals:  

1. understand customer behavior  
2. explain customer behavior to non-data staff  
3. optimize staffing so that the queues do not get unnecessary long  

## Supermarket Area

We are using the following model supermarket with six areas: entrance, fruit, spices, dairy, drinks and checkout.

The customers can move between these areas freely. Sooner or later, they will enter the checkout area. Once they do, they are considered to have left the shop.

![Drag Racing](./project/supermarket.png)

## 8.1. Data Analysis

### Load data

In [ ]:
import pandas as pd

In [1]:
from os import listdir
from os.path import isfile, join

# def load_file(day):
#     path = './project/data/'

#     df = pd.read_csv(os.path.join(path, day + '.csv'), sep=';', parse_dates=['timestamp'])
    
#     # individual dataframes with new column added to represent the day
#     df['day'] = day

#     return df

# df = load_file('monday').append(
#         load_file('tuesday').append(
#             load_file('wednesday').append(
#                 load_file('thursday').append(
#                     load_file('friday')
#                 )
#             )
#         )
# )
# df
def load_data(day):
    path = './project/data/'
    r = pd.read_csv(os.path.join(path, day + '.csv'), sep=';', parse_dates=['timestamp'], index_col=['timestamp'])
    r['day'] = day

#     r = r.head(3).copy().append(r.tail(3).copy()) # TODO: drop me

    return r

# files = [f for f in listdir(path) if isfile(join(path, f))]

# load first file
df = load_data('monday')

# join data from all remaining files
for file in ['tuesday', 'wednesday', 'thursday', 'friday']:

    df_next = load_data(file)
    df_next['customer_no'] = df_next['customer_no'] + df['customer_no'].max()

    df = df.append(df_next)

# df.reset_index(inplace=True, drop=True)
# df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df[(df.index > '2019-09-02 21:48:00') & (df.index < '2019-09-03 07:07:00')].sort_values(by='timestamp')

In [ ]:
# the total number of customers in each section (no unique customers)
df.groupby(by='location')['customer_no'].count()

## Fill out missing counter time

In [ ]:
# When the shop closes, the remaining customers are rushed through the checkout. 
# Their checkout is not recorded, so it may look as if they stay in the market forever.

# TODO: fill out missing counter rows

In [ ]:
# df.iloc[25]['location']

# def get_last_id(df):
#     return df.index[-1]

# df[:100].groupby('customer_no').agg({'location': get_last_id})

In [ ]:
# df.iloc[0:57]
df[df['customer_no'] == 6]

In [ ]:
# # df.pivot(columns=['location'], values=['timestamp'], index=['customer_no'])

# def get_last(df):
#     print(df)
#     return df.index[[0, -1]]

# df.iloc[0:56].pivot(index=['customer_no'], columns=['location'], values=['timestamp']) # df.iloc[0:57] will fail
# df.iloc[0:57].groupby(['customer_no', 'location']).agg({'location': get_last})
# df.iloc[0:57]

In [ ]:
# Calculate the total number of customers in each section over time

# Display the number of customers at checkout over time

In [ ]:
# The time each customer spent in the market
# visits = df.groupby(by='customer_no').index.agg(['min', 'max']) # TODO: find out how to aggregate by index value
# visits['duration'] = visits['max'] - visits['min']
# visits.sort_values(by='duration', ascending=False)

In [ ]:
# Calculate the total number of customers in the supermarket over time.

In [ ]:
# # Our business managers think that the first section customers visit follows a different pattern than the following ones. Plot the distribution of customers of their first visited section versus following sections (treat all sections visited after the first as “following”).

# df.groupby(['customer_no']).agg({'location': [' -> '.join, 'count']})

# df.groupby(['customer_no'])['location'].describe().sort_values(by='freq', ascending=False)

In [ ]:
# df.groupby(['customer_no'])['timestamp'].describe()
# # .sort_values(by='freq', ascending=False)

In [ ]:
### Probabilities plot

In [ ]:
# # initial_state = np.array([0.4, 0.6])  # e.g. cold, hot
# initial_state = []
# for column in crosstab.columns:
#     initial_state.append(int(column == 'entrance'))

In [ ]:
# state = initial_state
# lines = pd.DataFrame([state], columns=crosstab.columns)
# for i in range(0,20):
#     state = np.dot(state, crosstab.values)
#     lines = lines.append(pd.DataFrame([state], columns=crosstab.columns))
# lines = lines.reset_index()
# del lines['index']
# lines = lines.transpose()
# lines

In [ ]:
# for i in range(len(lines.columns)):
#     print('i: %d;  %0.4f' % (i, lines[i].sum()))

In [ ]:
# tmp = lines.transpose()
# for column in tmp.columns:
#     print('Column: %s, Sum: %0.2f' % (column, tmp[column].sum()))

In [ ]:
# tmp = lines.melt(value_vars=range(0, len(lines.columns)), var_name='step', ignore_index=False).reset_index().rename(columns={'index': 'section'})
# tmp

In [ ]:
# fig = px.line(tmp, x="step", y="value", color='section').show()

### Revenue Estimate

Estimate the total revenue for a customer using the following table:

| section | revenue per minute |
|---------|:--------------------:|
| fruit   | 4€                 |
| spices | 3€|
| dairy | 5€ |
| drinks | 6€ |

Which is the most profitable section according to your data?

## 8.2. Markov Chains

### Transition Probabilities

In [2]:
# find customers who visited several different sections
tmp = df.groupby('customer_no')['customer_no'].count()
tmp[(tmp > 3) & (tmp <6)].sample(n=3)
tmp = df[(df['customer_no'] == 3532) | (df['customer_no'] == 3685)].copy()

transitions = tmp.groupby(by=['customer_no']).resample('1T').pad().drop(columns=['customer_no']).reset_index()
transitions['location_before'] = transitions.groupby(by=['customer_no', 'day'])['location'].shift(fill_value='entrance')
transitions

,customer_no,timestamp,location,day,location_before
0,3532,2019-09-04 13:20:00,fruit,wednesday,entrance
1,3532,2019-09-04 13:21:00,dairy,wednesday,fruit
2,3532,2019-09-04 13:22:00,drinks,wednesday,dairy
3,3532,2019-09-04 13:23:00,drinks,wednesday,drinks
4,3532,2019-09-04 13:24:00,drinks,wednesday,drinks
5,3532,2019-09-04 13:25:00,drinks,wednesday,drinks
6,3532,2019-09-04 13:26:00,checkout,wednesday,drinks
7,3685,2019-09-04 14:48:00,fruit,wednesday,entrance
8,3685,2019-09-04 14:49:00,fruit,wednesday,fruit
9,3685,2019-09-04 14:50:00,dairy,wednesday,fruit


In [ ]:
# new df with column timestamp representing differences in first and last timestamp in the grouped table, ie. time spent in shop in minutes
# this is the time spent in the shop for each customer:
time_in_market = df.groupby(['customer_no'])[['timestamp']].last() - df.groupby(['customer_no'])[['timestamp']].first()
time_in_market

In [ ]:
# dummy column added :
time_in_market['counter'] = 1

# count the frequencies of the times spent in the shop:
time_in_market.groupby('timestamp').count().head()

In [ ]:
# Plot frequency distribution:
time_in_market.groupby('timestamp').count().plot()

In [ ]:
# df[(df['timestamp'] > '2019-09-02 21:48:00') & (df['timestamp'] < '2019-09-03 07:07:00')].sort_values(by='timestamp')

In [ ]:
df.groupby(by=['timestamp', 'customer_no']).last().reset_index().sort_values(by=['customer_no'])

In [ ]:
df.groupby(by=['customer_no', 'day'])['location'].shift(1).dropna()

In [ ]:
_ = df.groupby(by=['timestamp', 'customer_no', 'day'])['location'].last().reset_index()
_[_['timestamp'] > '2019-09-02 21:49:00']
# _['before'] = _['location'].shift(1)
# _['after'] = _['location']
# _[_['before'].isna()]

In [ ]:
# build a transition for an every minute
transitions = df.groupby(by=['timestamp', 'customer_no'])['location'].last().reset_index()
transitions['before'] = transitions['location'].shift(1)
transitions['before'].fillna('entrance', inplace=True)
transitions['after'] = transitions['location']
del transitions['location']
transitions[transitions['before'] == 'entrance']

# transitions = pd.DataFrame()
# transitions['customer_no'] = tmp['customer_no']
# transitions['before'] = tmp['location'].shift(1)
# transitions['after'] = tmp['location']
# transitions['before'].fillna('entrance', inplace=True)
# transitions

In [ ]:
crosstab = pd.crosstab(transitions['before'], transitions['after'], normalize=0)
# crosstab['entrance'] = 0
values = dict(zip(crosstab.columns, [0] * len(crosstab.columns)))
crosstab = crosstab.reindex(sorted(crosstab.columns), axis=1)
crosstab = crosstab.reindex(sorted(crosstab.index), axis=0)
crosstab

In [ ]:
crosstab.sum()

In [ ]:
crosstab.transpose()

In [ ]:
crosstab.transpose().sum()

### Probabilities plot

In [ ]:
# initial_state = np.array([0.4, 0.6])  # e.g. cold, hot
initial_state = []
for column in crosstab.columns:
    initial_state.append(int(column == 'entrance'))

In [ ]:
state = initial_state
lines = pd.DataFrame([state], columns=crosstab.columns)
for i in range(0,20):
    state = np.dot(state, crosstab.values)
    lines = lines.append(pd.DataFrame([state], columns=crosstab.columns))
lines = lines.reset_index()
del lines['index']
lines = lines.transpose()
lines

In [ ]:
for i in range(len(lines.columns)):
    print('i: %d;  %0.4f' % (i, lines[i].sum()))

In [ ]:
tmp = lines.transpose()
for column in tmp.columns:
    print('Column: %s, Sum: %0.2f' % (column, tmp[column].sum()))

In [ ]:
tmp = lines.melt(value_vars=range(0, len(lines.columns)), var_name='step', ignore_index=False).reset_index().rename(columns={'index': 'section'})
tmp

In [ ]:
px.line(tmp, x="step", y="value", color='section').show()

### MC-Simulation

In [ ]:
demo_transitions = np.array([[0.75, 0.4], [0.25, 0.6]])
demo_transitions

In [ ]:
demo_state0 = np.array([1, 0])
demo_state0

In [ ]:
demo_state1 = np.dot(demo_transitions, demo_state0)
demo_state1

In [ ]:
state2 = np.dot(demo_transitions, demo_state1)
state2

In [ ]:
ex = np.array([0.4, 0.6])
ex, type(ex)

In [ ]:
a = np.array([[1, 0],
              [0, 1]])
b = np.array([[4, 1],
              [2, 2]])
np.dot(a, b)

In [ ]:
import time
import datetime
datetime.datetime.strptime("01-12-2011", "%d-%m-%Y")

In [ ]:
np.datetime64('2005-02-25 18:25:01')

In [ ]:
_ = pd.DataFrame({
    'start': [
        np.datetime64('2005-02-25 18:23:01'),
        np.datetime64('2005-02-25 18:25:01'),
        np.datetime64('2005-02-25 18:25:01'),
        np.datetime64('2005-02-25 18:32:01'),
    ],
    'end': [
        np.datetime64('2005-02-25 18:25:01'),
        np.datetime64('2005-02-25 18:35:01'),
        np.datetime64('2005-02-25 18:32:01'),
        np.datetime64('2005-02-25 18:38:01'),
    ],
    'location': [
        'section1',
        'section1',
        'section2',
        'section2',
    ]
}, index=[0, 1, 2, 3])

In [ ]:
_['delta'] = _['end'] - _['start']
_

In [ ]:
_.index = _['start']
_.groupby('location').resample('1T').pad()

In [ ]:
type(_['delta'][0])

In [ ]:
full_idx = pd.date_range(start=_['start'].min(), end=_['end'].max(), freq='01T')
full_idx

In [ ]:
# df = df.apply(lambda group: group.reindex(full_idx, method='nearest')) 
# #  .groupby('location', as_index=False)  
# #  .apply(lambda group: group.reindex(full_idx, method='nearest')) 
# #  .reset_index(level=0, drop=True) 
# #  .sort_index() 

In [ ]:
data = {'datetime' : pd.date_range(start='1/15/2018',
                                  end='02/14/2018', 
                                  freq='D')\
                     .append(pd.date_range(start='1/15/2018',
                                           end='02/14/2018',
                                           freq='D')),
        'house' : ['house1' for i in range(31)] + ['house2' for i in range(31)],
        'readvalue' : [0.5 + 0.5*np.sin(2*np.pi/30*i) for i in range(31)] + [0.5 + 0.5*np.cos(2*np.pi/30*i) for i in range(31)]}

In [ ]:
# Randomly drop half the reads
import random

df0 = pd.DataFrame(data, columns = ('datetime', 
                                    'house', 
                                    'readvalue'))

random.seed(42)
df0 = df0.drop(random.sample(range(df0.shape[0]),
                             k=int(df0.shape[0]/2)))
df0

In [ ]:
def load_file(day):
    path = './project/data/'

    df = pd.read_csv(os.path.join(path, day + '.csv'), sep=';', parse_dates=['timestamp'])
    
    # individual dataframes with new column added to represent the day
    df['day'] = day

    return df

# dfs appended into one big df
total = load_file('monday').append(load_file('tuesday').append(load_file('wednesday').append(load_file('thursday').append(load_file('friday')))))

# new df with non-datetime index
time_ = total.reset_index()

#new df with column timestamp representing differences in first and last timestamp in the grouped table, ie. time spent in shop in minutes
# this is the time spent in the shop for each customer:
time_in_market = time_.groupby(['customer_no', 'day'])[['timestamp']].last() - time_.groupby(['customer_no', 'day'])[['timestamp']].first()
time_in_market

In [ ]:
# dummy column added :
time_in_market['counter'] = 1

# count number of customers at every second (count the frequencies of the times spent in the shop):
time_in_market.groupby('timestamp').count().head()

In [ ]:
# Plot frequency distribution:
# time_in_market.groupby('timestamp').count().plot()
_ = time_in_market.groupby('timestamp').count().reset_index()
_['minutes'] = _['timestamp'].dt.total_seconds() / 60
px.line(_, x="minutes", y="counter").show()

In [ ]:
listdir('./project/data/')